<a href="https://colab.research.google.com/github/JarekMaleszyk/data-scraping-examples/blob/main/quotes_to_postgresql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from bs4 import BeautifulSoup

In [2]:
quotes_url = "https://quotes.toscrape.com/"

response = requests.get(quotes_url)
response

<Response [200]>

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
quotes_list = []
for i in range(5):
    for quote in soup.find_all("div", class_="quote"):

        single_quote = dict()

        single_quote["text"] = quote.find("span", class_="text").text.replace('“', '').replace('”', '')

        single_quote["author"] = quote.find("small", class_="author").text

        tags = quote.find_all("a", "tag")
        tags_texts = [t.text for t in tags]
        single_quote["tags"] = tags_texts

        quotes_list.append(single_quote)

    sub_url = soup.find("li", class_="next").find("a").attrs["href"]
    next_page_url = quotes_url + sub_url
    response = requests.get(next_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
quotes_list[0]

{'text': 'The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.',
 'author': 'Albert Einstein',
 'tags': ['change', 'deep-thoughts', 'thinking', 'world']}

In [6]:
import psycopg2

In [7]:
DATABASE = "scrapy"
DB_PASSWORD = "postgres"
DB_USER = "postgres"
HOST = "localhost"
PORT = 5432

In [ ]:
conn = psycopg2.connect(dbname=DATABASE, user=DB_USER, password=DB_PASSWORD, host=HOST, port=PORT)

In [ ]:
cursor = conn.cursor()

In [ ]:
drop_query = '''
DROP TABLE IF EXISTS quotes;
'''

cursor.execute(drop_query)
conn.commit()

In [ ]:
create_query = '''
CREATE TABLE IF NOT EXISTS quotes (
    id          SERIAL PRIMARY KEY not null,
    text        TEXT,
    author      VARCHAR(255),
    tags        TEXT,
    insert_date DATE not null default CURRENT_DATE
);
'''

In [ ]:
cursor.execute(create_query)
conn.commit()

In [ ]:
import logging

try:
    for value in quotes_list:
        text = value["text"].replace("'", "''")
        author = value["author"]
        tags = ", ".join(value["tags"])
        insert_query =  f"""
                        INSERT INTO "quotes"
                        (id, "text", author, tags, insert_date)
                        VALUES(nextval('quotes_id_seq'::regclass), '{text}', '{author}', '{tags}', CURRENT_DATE);
                        """
        cursor.execute(insert_query)
except DatabaseError as dbe:
    logging.error(f"Database error: {dbe}")
    conn.rollback()
except ex:
    logging.error(f"Other error: {ex}")
    conn.rollback()
else:
    conn.commit()
finally:
    conn.close()
    cursor.close()